In [1]:
import os
import json
import io
from data_generator import DataGenerator
from trade_env import TraderEnv
import time

In [2]:
stage_length = 10

def get_enviroment(stage_length=10):
    return TraderEnv(DataGenerator(), stage_history_length=stage_length)

In [3]:
stage_length_var = 1
trade = get_enviroment(stage_length_var)
trade.reset()

array([ 1.        ,  1.00091519,  1.00091589,  1.00097681,  1.00114556,
        1.00123519,  1.00131782,  1.00131852,  1.00180868,  1.00185209,
        1.00204675,  1.00236885,  1.00255791,  1.00255861,  1.00256772,
        1.00257402,  1.00260133,  1.00275608,  1.00320912,  1.00326514,
       -0.99766616, -0.99766546, -0.99634414, -0.99634204, -0.99585888,
       -0.99513485, -0.99506903, -0.99506833, -0.99382404, -0.99382334,
       -0.99382123, -0.99254053, -0.9915154 , -0.99081518, -0.99011496,
       -0.98983487, -0.98941473, -0.98940143, -0.98871451, -0.98801429,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [4]:
for data_index in range(50000):
    next_state, reward, done, info  = trade.step(0)
    if done:
        print(reward, " ", info)
        break

-0.1   {'status': 'reward too low -500.1000000000452'}


In [ ]:
import threading
import numpy as np
import tensorflow as tf
import pylab
import time
import gym
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K


# global variables for threading
episode = 0
scores = []

EPISODES = 30000

# This is A3C(Asynchronous Advantage Actor Critic) agent(global) for the Cartpole
# In this example, we use A3C algorithm
class A3CAgent:
    def __init__(self, state_size, action_size, env_name):
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        
        episode = 0
        scores = []

        # get gym environment name
        self.env_name = env_name

        # these are hyper parameters for the A3C
        self.actor_lr = 0.00002
        self.critic_lr = 0.00002
        self.discount_factor = .6

        self.threads = 32

        # create model for actor and critic network
        self.actor, self.critic = self.build_model()

        # method for training actor and critic network
        self.optimizer = [self.actor_optimizer(), self.critic_optimizer()]

        self.sess = tf.InteractiveSession()
        K.set_session(self.sess)
        self.sess.run(tf.global_variables_initializer())

    # approximate policy and value using Neural Network
    # actor -> state is input and probability of each action is output of network
    # critic -> state is input and value of state is output of network
    # actor and critic network share first hidden layer
    def build_model(self):
        state = Input(batch_shape=(None,  self.state_size))
        shared = Dense(self.state_size*2, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_uniform')(state)
        
        actor_hidden = Dense(self.state_size, activation='relu', kernel_initializer='glorot_uniform')(shared)
        actor_hidden = Dense(128, activation='relu', kernel_initializer='glorot_uniform')(actor_hidden)
        actor_hidden = Dense(64, activation='relu', kernel_initializer='glorot_uniform')(actor_hidden)
        actor_hidden = Dense(32, activation='relu', kernel_initializer='glorot_uniform')(actor_hidden)
        actor_hidden = Dense(16, activation='relu', kernel_initializer='glorot_uniform')(actor_hidden)
        action_prob = Dense(self.action_size, activation='softmax', kernel_initializer='glorot_uniform')(actor_hidden)

        value_hidden = Dense(self.state_size, activation='relu', kernel_initializer='he_uniform')(shared)
        value_hidden = Dense(128, activation='relu', kernel_initializer='he_uniform')(value_hidden)
        value_hidden = Dense(64, activation='relu', kernel_initializer='he_uniform')(value_hidden)
        value_hidden = Dense(32, activation='relu', kernel_initializer='he_uniform')(value_hidden)
        value_hidden = Dense(16, activation='relu', kernel_initializer='he_uniform')(value_hidden)
        state_value = Dense(1, activation='linear', kernel_initializer='he_uniform')(value_hidden)

        actor = Model(inputs=state, outputs=action_prob)
        critic = Model(inputs=state, outputs=state_value)

        actor._make_predict_function()
        critic._make_predict_function()

        #actor.summary()
        #critic.summary()

        return actor, critic

    # make loss function for Policy Gradient
    # [log(action probability) * advantages] will be input for the back prop
    # we add entropy of action probability to loss
    def actor_optimizer(self):
        action = K.placeholder(shape=(None, self.action_size))
        advantages = K.placeholder(shape=(None, ))

        policy = self.actor.output

        good_prob = K.sum(action * policy, axis=1)
        eligibility = K.log(good_prob + 1e-10) * K.stop_gradient(advantages)
        loss = -K.sum(eligibility)

        entropy = K.sum(policy * K.log(policy + 1e-10), axis=1)

        actor_loss = loss + 0.01*entropy

        optimizer = Adam(lr=self.actor_lr)
        updates = optimizer.get_updates(self.actor.trainable_weights, [], actor_loss)
        train = K.function([self.actor.input, action, advantages], [], updates=updates)
        return train

    # make loss function for Value approximation
    def critic_optimizer(self):
        discounted_reward = K.placeholder(shape=(None, ))

        value = self.critic.output

        loss = K.mean(K.square(discounted_reward - value))

        optimizer = Adam(lr=self.critic_lr)
        updates = optimizer.get_updates(self.critic.trainable_weights, [], loss)
        train = K.function([self.critic.input, discounted_reward], [], updates=updates)
        return train

    # make agents(local) and start training
    def train(self):
        # self.load_model('./save_model/cartpole_a3c.h5')
        agents = [Agent(i, self.actor, self.critic, self.optimizer, self.env_name, self.discount_factor,
                        self.action_size, self.state_size) for i in range(self.threads)]

        for agent in agents:
            agent.start()

        for agent in agents:
            agent.join()

        plot = scores[:]
        pylab.plot(range(len(plot)), plot, 'b')
        pylab.savefig("./save_graph/model.png")

        self.save_model('./save_model/model')
        
        
    def save_model(self, name):
        self.actor.save_weights(name + "_actor.h5")
        self.critic.save_weights(name + "_critic.h5")

    def load_model(self, name):
        self.actor.load_weights(name + "_actor.h5")
        self.critic.load_weights(name + "_critic.h5")

# This is Agent(local) class for threading
class Agent(threading.Thread):
    def __init__(self, index, actor, critic, optimizer, env_name, discount_factor, action_size, state_size):
        threading.Thread.__init__(self)

        self.states = []
        self.rewards = []
        self.actions = []

        self.index = index
        self.actor = actor
        self.critic = critic
        self.optimizer = optimizer
        self.env_name = env_name
        self.discount_factor = discount_factor
        self.action_size = action_size
        self.state_size = state_size

    # Thread interactive with environment
    def run(self):
        global episode
        env = get_enviroment(stage_length_var)
        while episode < EPISODES:
            state = env.reset()
            score = 0
            
            while True:
                action = self.get_action(state)
                next_state, reward, done, _ = env.step(action)
                score += reward

                self.memory(state, action, reward)

                state = next_state

                if done or score < env.minimum_reward_limit:
                    episode += 1
                    #print("episode: ", episode, "/ score : ", score)
                    scores.append(score)
                    self.train_episode(score < 0)
                    break
        
    # In Policy Gradient, Q function is not available.
    # Instead agent uses sample returns for evaluating policy
    def discount_rewards(self, rewards, done=True):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        if not done:
            running_add = self.critic.predict(np.reshape(self.states[-1], (1, self.state_size)))[0]
        for t in reversed(range(0, len(rewards))):
            running_add = running_add * self.discount_factor + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    # save <s, a ,r> of each step
    # this is used for calculating discounted rewards
    def memory(self, state, action, reward):
        self.states.append(state)
        act = np.zeros(self.action_size)
        act[action] = 1
        self.actions.append(act)
        self.rewards.append(reward)

    # update policy network and value network every episode
    def train_episode(self, done):
        discounted_rewards = self.discount_rewards(self.rewards, done)

        values = self.critic.predict(np.array(self.states))
        values = np.reshape(values, len(values))

        advantages = discounted_rewards - values

        self.optimizer[0]([self.states, self.actions, advantages])
        self.optimizer[1]([self.states, discounted_rewards])
        self.states, self.actions, self.rewards = [], [], []

    def get_action(self, state):
        policy = self.actor.predict(np.reshape(state, [1, self.state_size]))[0]
        return np.random.choice(self.action_size, 1, p=policy)[0]


/home/samir/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [ ]:

discount_factor = 0
stage_history_length = 0

discount_factor_start = 0.1
stage_history_length_start = 10

actor_lr_max, critic_lr_max, discount_factor_max, stage_history_length_max = 0, 0, 0, 0

max_score = 0

max_config = ""

is_first = True

global_agent = A3CAgent(1, 2, "TraderEnv")

for discount_factor_x in range(3, 9):
    discount_factor = round(discount_factor_start + (discount_factor_x * discount_factor_start), 2)

    for stage_history_length_x in range(1, 4):
        stage_history_length = int(stage_history_length_start + (stage_history_length_x * stage_history_length_start))
        
        stage_length_var=stage_history_length

        env = TraderEnv(DataGenerator(), stage_history_length=stage_history_length)

        state_size = env.observation_space.shape[0]
        action_size = env.action_space.n

        global_agent = A3CAgent(state_size, action_size, "TraderEnv")
        
        global_agent.discount_factor = discount_factor
        
        config = "discount_factor %f stage_history_length %s " % (global_agent.discount_factor, stage_history_length)

        print("#############################")
        print(config)
        print("#############################")
        
        global_agent.train()
        
        print("#############################")
        print("Trainning finished")
        print("#############################")
        
        def get_trained_score(data_index, stage_history_length):

            env = TraderEnv(DataGenerator(random=False, first_index=data_index), stage_history_length=stage_history_length)

            def get_action(state):
                policy =  global_agent.actor.predict(np.reshape(state, [1, state_size]))[0]
                return np.argmax(policy)

            state = env.reset()
            score = 0
            while True:
                action = get_action(state)
                #print(action)
                next_state, reward, done, info = env.step(action)
                score += reward

                state = next_state
                if done:
                    print ("score: ", score , "info", info)
                    break
            return score
        
        score = 0
              
        for data_index in range(1, 3):
            score += get_trained_score(1000*data_index, stage_history_length)
        
        print("#############################")
        print("%s: %s " % (config, score))
        print("#############################")
        if score > max_score or is_first:
            is_first = False
            max_score = score
            max_config = config
            discount_factor_max, stage_history_length_max = discount_factor, stage_history_length
            print("Record: %s: %s " % (config, score))
            global_agent.save_model('./save_model/best')


#############################
discount_factor 0.800000 stage_history_length 30 
#############################


In [ ]:
print (max_config, " :: ", max_score)

In [ ]:
env = TraderEnv(DataGenerator(), stage_history_length=30)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n


In [ ]:
state_size, action_size

In [ ]:
global_agent = A3CAgent(state_size, action_size, "TraderEnv")
global_agent.train()

In [ ]:
def get_action(state):
    policy =  global_agent.actor.predict(np.reshape(state, [1, state_size]))[0]
    return np.argmax(policy)

In [ ]:
discount_factor = 0.900000 
stage_history_length = 40

In [ ]:

env = TraderEnv(DataGenerator(random=False, first_index=data_index), stage_history_length=stage_history_length)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n

global_agent = A3CAgent(state_size, action_size, "TraderEnv")

global_agent.discount_factor = discount_factor


In [ ]:
global_agent.load_model('./save_model/model')

In [ ]:
score = 0
def get_trained_score(data_index, stage_history_length):
    print (data_index)
    env = TraderEnv(DataGenerator(random=False, first_index=data_index), stage_history_length=stage_history_length)

    def get_action(state):
        policy =  global_agent.actor.predict(np.reshape(state, [1, state_size]))[0]
        return np.argmax(policy)

    state = env.reset()
    score = 0
    while True:
        action = get_action(state)
        #print(action)
        next_state, reward, done, info = env.step(action)
        score += reward

        state = next_state
        if done:
            print ("score: ", score , "info", info)
            break
    return score


In [ ]:
for data_index in range(1, 3):
    score += get_trained_score(100*data_index, stage_history_length)

In [ ]:
round(0.30000000000000004, 2)